# This notebook explores the results for the agent based model

In [1]:
import re
import glob
import json
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option( 'display.max_columns', None )
pd.set_option( 'display.max_rows', None )

# Loading localidades GeoJSON for reference

# Loading ABM results

In [21]:
escenario = 'localidades'

city = 'bogota'
scenario = 'sin-cierres'
scenario_name = 'Sin cierres'
init_date = pd.to_datetime( '03/06/2020' )

In [22]:
path = './data/agents/{}/{}/{}/*.csv'.format( escenario, city, scenario )

In [23]:
files = glob.glob( path )

In [24]:
files.sort()

In [25]:
agents = []
exp = 0

for file in files:
    df = pd.read_csv( file )
    df[ 'Fecha' ] = df[ 'Día' ].apply( lambda x: init_date + pd.DateOffset( days = x ) )
    df[ 'exp' ] = exp
    agents.append( df )
    exp = exp + 1

agents = pd.concat( agents )
del agents[ 'Unnamed: 0' ]

In [26]:
agents.shape

(11200, 44)

In [27]:
agents.dtypes

Susceptibles                                   int64
Total infectados                               int64
Graves                                         int64
Críticos                                       int64
Leves                                          int64
Recuperados                                    int64
Rt                                           float64
Recuento_zonas                                object
0-9                                           object
10-19                                         object
20-39                                         object
40-59                                         object
>60                                           object
En_cuarentena                                 object
Restringido_movilidad                         object
Vivos                                          int64
Día                                            int64
Contactos_prom_trabajo                       float64
Contactos_prom_transporte                    f

In [28]:
agents.tail()

,Susceptibles,Total infectados,Graves,Críticos,Leves,Recuperados,Rt,Recuento_zonas,0-9,10-19,20-39,40-59,>60,En_cuarentena,Restringido_movilidad,Vivos,Día,Contactos_prom_trabajo,Contactos_prom_transporte,Contactos_prom_casa,Contactos_prom_trabajo_conm,Contactos_prom_transporte_conm,Contactos_prom_casa_conm,Nuevos_infectados,Detectados,En_testing,En_cama,En_UCI,Detectados_por_intervencion,#Intervenidos,porcentaje_edad_activa_por_zona,porcentaje_edad_activa,tasa_letalidad_por_zona,tasa_letalidad,total_muertos,total_muertos_por_zona,total_muertos_edad_activa,total_muertos_edad_activa_por_zona,zonas intervenidas,Recuento_actividades,Graves_efectivos,Criticos_efectivos,Fecha,exp
345,22600,9,0,0,9,22530,0.555556,"{11: [3156, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3...","[3003, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2392, ...","[3644, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 2928, ...","[7755, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 8295, ...","[5242, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6055, ...","[2956, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2860, ...","[0, 0, 0, 0, 0]","[10658, 4, 0, 0, 10159]",45139,345,0.546667,1.538470,2.539999,1.015264,2.857231,2.570088,1,545,0,42,4,0,0,"{1: 0.0033102618116160097, 2: 0.0, 3: 0.0, 4: ...",0.007789,"{1: 0.2, 2: 0.0, 3: 0, 4: 0.2, 5: 0, 6: 0.15, ...",0.09015,54,"{1: 5, 2: 0, 3: 0, 4: 2, 5: 0, 6: 3, 7: 4, 8: ...",6,"{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 2, 8: ...","{8: [[87, 13], [88, 13], [89, 13], [90, 13], [...","{5: [1100, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 12...",0,0,2021-02-14,31
346,22600,8,0,0,8,22531,0.625000,"{11: [3156, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3...","[3003, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2392, ...","[3644, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2929, ...","[7755, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 8295, ...","[5242, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6055, ...","[2956, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2860, ...","[0, 0, 0, 0, 0]","[11587, 6, 0, 0, 10792]",45139,346,0.489200,1.195175,2.526374,0.971064,2.372427,2.553034,0,545,0,42,4,0,0,"{1: 0.0033102618116160097, 2: 0.0, 3: 0.0, 4: ...",0.007789,"{1: 0.2, 2: 0.0, 3: 0, 4: 0.2, 5: 0, 6: 0.15, ...",0.09015,54,"{1: 5, 2: 0, 3: 0, 4: 2, 5: 0, 6: 3, 7: 4, 8: ...",6,"{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 2, 8: ...","{8: [[87, 13], [88, 13], [89, 13], [90, 13], [...","{5: [1100, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 12...",0,0,2021-02-15,31
347,22598,10,0,0,10,22531,0.700000,"{11: [3155, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 3...","[3003, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2392, ...","[3643, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2929, ...","[7754, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 8295, ...","[5242, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6055, ...","[2956, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2860, ...","[0, 0, 0, 0, 0]","[12033, 7, 0, 0, 11383]",45139,347,0.452270,1.105075,2.520858,0.940567,2.298180,2.535222,1,545,0,42,4,0,0,"{1: 0.0033102618116160097, 2: 0.0, 3: 0.0, 4: ...",0.007789,"{1: 0.2, 2: 0.0, 3: 0, 4: 0.2, 5: 0, 6: 0.15, ...",0.09015,54,"{1: 5, 2: 0, 3: 0, 4: 2, 5: 0, 6: 3, 7: 4, 8: ...",6,"{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 2, 8: ...","{8: [[87, 13], [88, 13], [89, 13], [90, 13], [...","{5: [1100, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 12...",0,0,2021-02-16,31
348,22597,10,0,0,10,22532,0.700000,"{11: [3155, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 3...","[3002, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2392, ...","[3643, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 2929, ...","[7754, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 8296, ...","[5242, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6055, ...","[2956, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2860, ...","[0, 0, 0, 0, 0]","[10277, 5, 0, 0, 9975]",45139,348,0.566849,1.603248,2.539290,1.028913,2.910125,2.551391,1,545,0,42,4,0,0,"{1: 0.0033102618116160097, 2: 0.0, 3: 0.0, 4: ...",0.007789,"{1: 0.2, 2: 0.0, 3: 0, 4: 0.2, 5: 0, 6: 0.15, ...",0.09015,54,"{1: 5, 2: 0, 3: 0, 4: 2, 5: 0, 6: 3, 7: 4, 8: ...",6,"{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 2, 8: ...","{8: [[87, 13], [88, 13], [89, 13], [90, 13], [...","{5: [1100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12...",0,0,2021-02-17,31
349,22596,10,0,0,10,22533,

In [29]:
def sum_quarantine( x ):
    x = json.loads( x )
    people_in_q = x[ 0 ] + x[ 1 ] + x[ 2 ] + x[ 3 ] + x[ 4 ]
    return people_in_q

In [30]:
agents_norm = []

for exp in agents[ 'exp' ].unique():
    df = agents.loc[ agents[ 'exp' ] == exp ]
    total_people = df[ 'Vivos' ].values[ 0 ]
    df[ 'Graves' ] = df[ 'Graves' ] / total_people
    df[ 'Críticos' ] = df[ 'Críticos' ] / total_people
    df[ 'Fallecidos' ] = ( total_people - df[ 'Vivos' ] ) / total_people
    df[ 'R0' ] = ( df[ 'Nuevos_infectados' ].cumsum() / total_people )
    df[ 'Cuarentena' ] = df[ 'En_cuarentena' ].apply( lambda x: sum_quarantine( x ) ) / total_people
    agents_norm.append( df[ [ 'exp', 'Fecha', 'Graves', 'Críticos', 'Fallecidos', 'R0', 'Rt', 'Cuarentena' ] ] )

agents_norm = pd.concat( agents_norm )

/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [31]:
agents_norm.tail()

,exp,Fecha,Graves,Críticos,Fallecidos,R0,Rt,Cuarentena
345,31,2021-02-14,0.0,0.0,0.001195,0.499790,0.555556,0.0
346,31,2021-02-15,0.0,0.0,0.001195,0.499790,0.625000,0.0
347,31,2021-02-16,0.0,0.0,0.001195,0.499812,0.700000,0.0
348,31,2021-02-17,0.0,0.0,0.001195,0.499834,0.700000,0.0
349,31,2021-02-18,0.0,0.0,0.001195,0.499856,0.600000,0.0


In [32]:
var = 'Críticos'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}/{}-{}-{}.csv'.format( escenario, city, city, scenario, 'ucis' ), index = False )

var = 'Fallecidos'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}/{}-{}-{}.csv'.format( escenario, city, city, scenario, 'deaths' ), index = False )

var = 'R0'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}/{}-{}-{}.csv'.format( escenario, city, city, scenario, 'r0' ), index = False )

var = 'Rt'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}/{}-{}-{}.csv'.format( escenario, city, city, scenario, 'rt' ), index = False )

var = 'Cuarentena'
agents_norm[ [ 'exp', 'Fecha', var ] ].rename( columns = { var: scenario_name } ) \
    .set_index( [ 'exp', 'Fecha' ] ).stack().reset_index().rename( columns = { 'level_2': 'variable', 0: 'value' } ) \
    .to_csv( './agents-viz-2/data/{}/{}/{}-{}-{}.csv'.format( escenario, city, city, scenario, 'quarantine' ), index = False )

In [ ]:
def ci( x ):
    try:
        return 1.96 * np.std( x ) / np.sqrt( len( x ) )
    except ZeroDivisionError:
        return 0

agents_norm_grouped = agents_norm.groupby( 'Fecha' ).agg( { 'Graves': [ 'mean', ci ], 'Críticos': [ 'mean', ci ], 'Fallecidos': [ 'mean', ci ], 'R0': [ 'mean', ci ], 'Rt': [ 'mean', ci ], 'Cuarentena': [ 'mean' ] } )
agents_norm_grouped.columns = [ '_'.join( col ).strip() for col in agents_norm_grouped.columns.values ]

In [ ]:
agents_norm_grouped.tail()

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Graves_mean' ], label = 'Graves', color = '#317372' )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Graves_mean' ] - agents_norm_grouped[ 'Graves_ci' ] ), ( agents_norm_grouped[ 'Graves_mean' ] + agents_norm_grouped[ 'Graves_ci' ] ), color = '#317372', alpha = .1 )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Críticos_mean' ], label = 'Críticos', color = '#f98a4b' )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Críticos_mean' ] - agents_norm_grouped[ 'Críticos_ci' ] ), ( agents_norm_grouped[ 'Críticos_mean' ] + agents_norm_grouped[ 'Críticos_ci' ] ), color = '#f98a4b', alpha = .1 )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Fallecidos_mean' ], label = 'Fallecidos', color = '#f64438' )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Fallecidos_mean' ] - agents_norm_grouped[ 'Fallecidos_ci' ] ), ( agents_norm_grouped[ 'Fallecidos_mean' ] + agents_norm_grouped[ 'Fallecidos_ci' ] ), color = '#f64438', alpha = .1 )

plt.title( 'Graves / Críticos / Fallecidos' )
plt.xlabel( 'Fecha' )
plt.ylabel( 'Casos' )
plt.grid( True )
plt.legend()

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'R0_mean' ] )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'R0_mean' ] - agents_norm_grouped[ 'R0_ci' ] ), ( agents_norm_grouped[ 'R0_mean' ] + agents_norm_grouped[ 'R0_ci' ] ), alpha = .1 )

plt.title( 'Tasa de ataque' )
plt.xlabel( 'Fecha' )
plt.ylabel( 'R0' )
plt.grid( True )

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Rt_mean' ] )
ax.fill_between( agents_norm_grouped.index, ( agents_norm_grouped[ 'Rt_mean' ] - agents_norm_grouped[ 'Rt_ci' ] ), ( agents_norm_grouped[ 'Rt_mean' ] + agents_norm_grouped[ 'Rt_ci' ] ), alpha = .1 )

plt.title( 'Tasa de reproducción afectiva' )
plt.xlabel( 'Fecha' )
plt.ylabel( 'Rt' )
plt.grid( True )

In [ ]:
fig, ax = plt.subplots( figsize = ( 20, 7 ) )

ax.plot( agents_norm_grouped.index, agents_norm_grouped[ 'Cuarentena_mean' ] )

plt.title( 'Cuarentena' )
plt.xlabel( 'Fecha' )
plt.ylabel( '% cuarentena' )
plt.grid( True )

In [ ]:
agents_norm.tail()

In [ ]:
agents_norm[ [ 'exp', 'Fecha', 'Graves', 'Críticos', 'Fallecidos', 'R0', 'Rt', 'Cuarentena' ] ].to_csv( './data/agents/clean/{}-{}.csv'.format( city, scenario ), index = False )